In [17]:
%load_ext autoreload
%autoreload 2
from components import component, event_handler, rpc, EventBroadcaster, ReceiverChannel, create_component_starter, loop


@component(["from_MyTestComponent1"])
class MyTestComponent1:
    def __init__(self, from_MyTestComponent1:EventBroadcaster):
        self.from_MyTestComponent1 = from_MyTestComponent1

    def step(self):
        self.from_MyTestComponent1.publish("hello Event 1")

    @classmethod
    def shared_samples(cls, manager):
        return [], []
        
    @rpc
    def myrpc(self, msg):

        print(f'com1 received msg: {msg}')
        return msg#"returned from com1"

    @event_handler("from_MyTestComponent2")
    def recv(self, msg):
        print(f'event:from_MyTestComponent2, received by com1: {msg}')



@component(["from_MyTestComponent2"])
class MyTestComponent2:
    def __init__(self, from_MyTestComponent2:EventBroadcaster, component1call: ReceiverChannel):
        self.from_MyTestComponent2 = from_MyTestComponent2
        self.component1call = component1call 
        

    def step(self):
        pass


    @rpc
    def by_main_event(self, msg):

        self.from_MyTestComponent2.publish("event from com2")

    @rpc
    def by_main_rpc1(self, msg):

        res = self.component1call.send_and_await_response("message from com2")

        return res



    @classmethod
    def shared_samples(cls, manager):
        return [], []



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from multiprocessing import Manager
m = Manager()

In [19]:
starter1, outgoing_value_samplers1, incoming_rpc1, outgoing_event_broadcasters1 = create_component_starter(

    MyTestComponent1, manager=m, loop=loop, init_kwargs={}, loop_kwargs={'ideal_interval': 1}, instantiater=None
)
starter2, outgoing_value_samplers2, incoming_rpc2, outgoing_event_broadcasters2 = create_component_starter(

    MyTestComponent2, manager=m, loop=loop, init_kwargs={}, loop_kwargs={'ideal_interval': 1}, instantiater=None
)



In [20]:
incoming_rpc1['myrpc'].send_and_await_response(np.zeros((1,3)))

KeyboardInterrupt: 

In [22]:
incoming_rpc1['myrpc'].msg_pending[:]

KeyboardInterrupt: 

In [21]:
incoming_rpc2

{'by_main_event': <components.ReceiverChannel at 0x76869dc2ed10>,
 'by_main_rpc1': <components.ReceiverChannel at 0x76869dc2cb20>}

In [23]:
l = m.list()

In [24]:
l.append(np.zeros((1,3)))

KeyboardInterrupt: 

In [12]:

p1 = starter1(outgoing_value_samplers2, {}, outgoing_event_broadcasters2)

p2 = starter2(outgoing_value_samplers1, {"component1call": incoming_rpc1['myrpc']}, outgoing_event_broadcasters1)


In [6]:
import numpy as np

In [14]:
incoming_rpc2['by_main_rpc1'].send_and_await_response(np.zeros((1,3)))

com1 received msg: message from com2
com1 received msg: [[0. 0. 0.]]
com1 received msg: [[0. 0. 0.]]


'returned from com1'

In [13]:
incoming_rpc2[1].send_and_await_response('from main 1')

KeyError: 1

In [7]:
incoming_rpc2[0].send_message('test')
incoming_rpc2[0].msg_pending[:]

[(1, 'test')]

In [8]:
incoming_rpc2[0].msg_pending[:]

[]

In [175]:
e = EventBroadcaster('test', m)
sub = e.subscrible("test_sub")

In [196]:
sub.start_message_handling_thread(lambda x: x, True)

<Thread(Thread-223 (message_handling_loop), started 131259854476864)>

In [212]:
sub.send_and_await_response('hi')

In [149]:
sub.send_and_await_response('hifasdfasd')

'hifasdfasd'

In [155]:
e.publish('h')

In [172]:
sub.send_message('gsd')

(14, None)

In [167]:
e.publish('r')
sub.await_message()

KeyboardInterrupt: 

In [174]:
sub.msg_pending[:]

[(13, 'gsd'), (14, 'gsd')]

In [170]:
sub.msg_semaphore.release()

Exception in thread Thread-163 (message_handling_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 282, in message_handling_loop
    self.await_and_handle_message(handler, to_reply)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 274, in await_and_handle_message
    msg_id, msg = self.await_message()
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 255, in await_mess

In [161]:
sub.msg_pending[:]

[]

In [162]:
sub.await_message()

KeyboardInterrupt: 

In [16]:
import time
import threading

In [6]:
sub.response_locks.keys()

[]

In [88]:
def test():
    for i in range(10):
        msg_id, lock = sub.send_message(i, True)
        print(f"sending {msg_id}")
        time.sleep(1)

threading.Thread(target=test).start()

sending 0
sending 1
sending 2
sending 3
sending 4
sending 5
sending 6
sending 7
sending 8
sending 9


In [128]:
sub.response_locks.get(1)

<AcquirerProxy object, typeid 'Lock' at 0x7761535b6860>

In [129]:
sub.reply(4, 'res')

4


In [93]:


sub.await_and_handle_message(lambda x: x, True)

ValueError: a response is not expected for this msg_id: 4

In [73]:
sub.msg_id

52

In [82]:
t = sub.start_message_handling_thread(lambda x: print(x), to_reply=True)

Exception in thread Thread-105 (message_handling_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 277, in message_handling_loop
    self.await_and_handle_message(handler, to_reply)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 272, in await_and_handle_message
    self.reply(msg_id, res)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 262, in reply
    raise Value

7


In [138]:


class T:
    def __init__(self): 
        self.dp = m.dict()
        self.d = {}

    def th_set(self, v):
        self.dp['t'] = v
        self.d['t'] = v

    def th_get(self):
        while True:
            time.sleep(1)
            print(self.d['t'])
            print(self.dp['t'])
    def test_th_set(self, v):
        threading.Thread(target=t.th_set, args=(v, )).start()

    def test_th_get(self):
        threading.Thread(target=t.th_get).start()



t = T()
t.test_th_set('1sdsds23')


In [140]:
t.

In [139]:
t.test_th_get()

1sdsds23
1sdsds23
1sdsds23
1sdsds23
1sdsds23
1sdsds23
1sdsds23
1sdsds23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf
1sd23
eadfsdf


In [141]:
t.dp['t']="eadfsdf"

In [104]:
threading.Thread(target=t.th_set, args=('1231aaa2', )).start()


In [102]:
t.th_get()

12312
12312


In [113]:
threading.Thread(target=t.th_get).start()


asdasd
asdasd


In [51]:
sub.send_message('hsi', True)

Exception in thread Thread-67 (message_handling_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 277, in message_handling_loop
    self.await_and_handle_message(handler, to_reply)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 272, in await_and_handle_message
    self.reply(msg_id, res)
  File "/home/kawa/CurrentProjects/pi_projects/self-driving/remote-control-car-1/src/components/__init__.py", line 262, in reply
    raise ValueE

hsi


(5, <AcquirerProxy object, typeid 'Lock' at 0x7dbe7d7f0e20>)

In [41]:
sub.response_locks[1] is None

False

In [49]:
sub.message_handling_loop(lambda x: print(x), to_reply=True)

hsi


KeyboardInterrupt: 

In [28]:
from multiprocessing import Semaphore

s = Semaphore(0)

In [29]:
s.release()

In [31]:
s.acquire()

KeyboardInterrupt: 

In [13]:
msg_id, lock = sub.send_message('hello', True)

In [41]:
from multiprocessing import Lock
l = Lock()

In [42]:
d = {}
d[1] = l

In [45]:
class A:
    def __init__(self):
        self.d = {}

    def nl(self, k):
        l = Lock()
        self.d[k] = l

a  = A()
a.nl('e')

In [46]:
a.d['e']

<Lock(owner=None)>

In [14]:
sub.await_and_handle_message(lambda x: x, True)

'hello'

In [15]:
sub.await_response(msg_id)

'hello'

In [ ]:
sub.start_message_handling_thread)